# Time Series Analysis

The exercise this week is to work with a time series data set to try to predict the future values from those in the past.  The procedure was outlined in the lecture last week.  The dataset we will use is of measures of average carbon dioxide ($CO_2$) concentration taken each month from 1958 onwards, hosted [here](https://www.esrl.noaa.gov/gmd/ccgg/trends/data.html) by the US National Oceanic & Atmospheric Administration.  

The data shows a clear upward trend over time with seasonal oscillations.  The first task is to load the data into a Pandas time series.  You should have a look at [the data file](files/co2_mm_mlo.txt).  Note that this data is available as a text file with whitespace delimeters between columns and that there are comment lines at the start of the file that we must skip. I have to provide the column names explicitly.  NA values in the data are encoded as -1, so we use the `na_values` argument to `read_table` to recognise these. 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.tsa.arima_model import ARIMA
from sklearn import linear_model

%matplotlib inline

In [ ]:
# set default figure size
plt.rcParams['figure.figsize'] = 15, 6

In [ ]:
co2 = pd.read_table('files/co2_mm_mlo.txt', delim_whitespace=True, comment="#", 
                    names=['year', 'month', 'decdate', 'average', 'interp', 'trend', 'days'],
                   na_values="-1")
co2.head()

We then create a date index, since the data includes just year and month colums, we add a constant `day` column to use in creating the datetime index.  We then remove the columns from the dataframe that aren't relevant (all except 'average') and drop any rows that contain NaN values. 

In [ ]:
co2['day'] = 1
co2.index = pd.to_datetime(co2[['year', 'month', 'day']])
co2.drop(['day', 'days', 'interp', 'trend', 'decdate', 'year', 'month'], axis=1, inplace=True)
co2.dropna(inplace=True)
co2.head()

Note that the data frame is not evenly spaced - there are some months that don't have any data.   To properly analyse the time series we need it to be evenly spaced, so we use the `resample` method to create a new version.  The `bfill` method here creates a new time series with unknown values _back filled_ from the next time point.  Observe the effect on the data for 1958-06-01 here that is missing in the original.

In [ ]:
# resample to get even frequency - fill unknown values
co2rs = co2.resample('MS').bfill()
co2rs.head()

Next we make a column `elapseddays` to contain a count of the number of days from the start for each row. This will be used as our `x` value for modelling later on. 

In [ ]:
# create a column with the number of days since the first sample
co2rs['elapseddays'] = (co2rs.index - co2rs.index[0]).days
co2rs.head()

## Your Task

Ok, now you take over.  Your goal is to first make the time series stationary, then to model the transformed time-series using an ARIMA model.   A strong hint is that there is a clear trend line in the data so computing a rolling mean that you then subtract from the `average` column is the way to go.

First, test the time-series for stationarity using the functions defined in the lecture and reproduced here.  Then compute the rolling mean, subtract it from the average column and repeat the test on these values.

In [ ]:
def plot_stationarity(timeseries, window=12):
    """Generate rolling plots of the time series"""
    
    #Determing rolling statistics
    rolmean = timeseries.rolling(window=window,center=False).mean() 
    rolstd = timeseries.rolling(window=window,center=False).std() 

    #Plot rolling statistics:
    orig = plt.plot(timeseries, color='blue',label='Original')
    mean = plt.plot(rolmean, color='red', label='Rolling Mean')
    std = plt.plot(rolstd, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show(block=False)

def test_stationarity(timeseries, window=12):
    """Run the Dickey-Fuller test for stationarity"""
  
    #Perform Dickey-Fuller test:
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], 
                         index=['Test Statistic','p-value',
                                '#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    return dfoutput

In [ ]:
plot_stationarity(co2rs['average'])
test_stationarity(co2rs['average'])

In [ ]:
# compute the rolling mean of the 'average' column


In [ ]:
# The rolling mean introduces some NaN values at the start
# use the dropna method to remove the rows containing NaN values



In [ ]:
# subtract the rolling mean from the average to give a normalised time series, add it as a new column to the data frame



In [ ]:
# test the stationarity of the new time series


## Fit ARIMA Model

Your next task is to fit an ARIMA model to the normalised column.  First we split the data into training and testing partitions.  We use the first 650 points for trianing and the remainder for testing.   Note that we use the copy method for the test data to get a copy rather than a view on the original data frame since we will want to add new columns for our predictions later. 

Esitmate the order of model that you should use (coefficients $p$ and $q$) and then build and train an ARIMA model. Plot the fitted values overlayed with the orginal data.

In [ ]:
# split into training and testing data
train = co2rs.iloc[:650]
test = co2rs.iloc[650:].copy()  # copy this because we want to add to it later

In [ ]:
# train an ARIMA model
armodel = 

---
Now that you have a model you can make predictions for the unknown time points in the test data and compare these to the true values.  To do this you can use the [`forecast` method of the ARIMA model](https://www.statsmodels.org/dev/generated/statsmodels.tsa.arima_model.ARMAResults.forecast.html) which returns three arrays (forecast, error, confidence).   

In [ ]:
# forecast as many points as are in our test data
forecast, error, conf = armodel.forecast(steps=test.average.size)
# add them to the data frame and plot
test['forecast'] = forecast

In [ ]:
# plot the actual and forecast values together


## Linear model of trend over time

Next we look at the rolling mean of the original average co2 time series which was subtracted from the original to allow us to fit the ARIMA model.  We can see that this is a gradual increase over time so we could predict future values using a linear model.  

Fit a linear model to predict `rolling` from `elapseddays` on the training data. Use it to predict future values for the test data, add these values as a new column to the test data frame. 

In [ ]:
# plot the rolling mean vs elapseddays


In [ ]:
# build a linear model and predict the co2 average for each of the values in elapseddays in the test data


## Combined Prediction

Now that we have predictions for both the overall trend and the seasonal variation in CO2 average they can be combined to give an overall prediction for the CO2 values in the range of the test data.  Compute this value and plot the true and predicted average values.